In [1]:
################################################################################
# MIT License
#
# Copyright (c) 2021 University of Amsterdam
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to conditions.
#
# Author: Deep Learning Course (UvA) | Fall 2022 & Oliver Gurney-Champion | Spring 2023
# Date modified: Jan 2023
################################################################################


# Import requiered packages
imports the packages and sets the random seed

In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import random
import numpy as np
import helper_functions as hf
import torch
from matplotlib import pyplot as plt

# set random seed
seed =42

login to your free wandb account. Note you will need to set up your account on https://wandb.ai/authorize
wandb allows you to keep track of your neural network training.

In [9]:
import wandb
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

# Simulate and view the IVIM data
Here, we split our data into a training set, validation set and test set. Note that the current implementation only uses the training set and it is up to you (in your exercises) to also implement the validation and test run. At this point, we already split the data for you.

In [28]:
# set b-values at which we "measure" (i.e. simulate signal)
bvalues=[0, 10, 20, 30, 50, 75, 100, 150, 300, 500, 700, 1000]

## Set the random seeds for reproducibility
np.random.seed(seed)
random.seed(seed)

#determine
batch_size = 128

with wandb.init(project="AI_for_medical_imaging", job_type="visualize data") as run:
    data_sim = hf.sim_signal(SNR=(10,40),bvalues=bvalues,sims=10000,seed=np.random.randint(1,10000))
    # Only for visualisation purposes: here we create our "Artifact" in wandb --> this allows viewing the data in your wandb account
    for i in range(4):
        #make b-value data pairs
        example_data=[[x,y] for (x,y) in zip(bvalues,data_sim[0][i])]
        # put it in a table
        table = wandb.Table(data=example_data, columns=["b-values", "signal"])
        #tell wandb to plot the table
        wandb.log({"data_plot " + str(i): wandb.plot.scatter(table, "b-values", "signal")})

    # here we split the data into train (70%), test (15%) and validation (15%) sets
    #split = int(np.floor(len(data_sim[0]) * 0.7))
    train_set, test_set, val_set = torch.utils.data.random_split([[data_sim[0][i,:],data_sim[1][i],data_sim[2][i],data_sim[3][i]] for i in range(len(data_sim[3]))],[0.7,0.15,0.15])
    #split = int(np.floor(len(rest) * 0.5))
    #test_set, val_set = torch.utils.data.random_split([[rest[0][i,:],rest[1][i],rest[2][i],rest[3][i]] for i in range(len(rest[3]))],[split, len(rest[0]) - split])

    # train loader loads the trianing data. We want to shuffle to make sure data order is modified each epoch and different data is selected each epoch.
    trainloader = torch.utils.data.DataLoader(train_set,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   drop_last=True)
    # validation data is loaded here. By not shuffling, we make sure the same data is loaded for validation every time. We can use substantially more data per batch as we are not training.
    inferloader = torch.utils.data.DataLoader(val_set,
                                   batch_size=batch_size,
                                   shuffle=False,
                                   drop_last=True)
        # validation data is loaded here. By not shuffling, we make sure the same data is loaded for validation every time. We can use substantially more data per batch as we are not training.
    testloader = torch.utils.data.DataLoader(test_set,
                                   batch_size=batch_size,
                                   shuffle=False,
                                   drop_last=True)

error/random error,███████████████▃▁▁█████▇▆▁▁█████████████
error/systematic error,▁▅▇▇██████████▇██▇▇▇████▇▇█▇▇▇██████▁▅▆▇
loss/train,█▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▁▃▃▃▃▃▃▃▃▁▁█▄▃▃▃▃▃▃▃▃▃▃▃
loss/val,█▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▄▃▁▁▅▅▅▅▅▅▅▅▃▅▅▅▅▅▅▅▅▅▅
error/random error,0.19508
error/systematic error,-0.00953
loss/train,0.04119
loss/val,0.04018


3 A.	Program the different neural network modules (Lineairemodule, Relumodule, Tanhmodule, MSE) in “modules.py”. Use “do_unittest.py” to test whether the forward and backward passes are correct. Make only use of matrix multiplications, no for loops. Points will be deduced when for-loops are used where matrix multiplications were possible.

3 B.  Combine all modules into a multilayer perceptron (MLP) in “exercise_3.ipynb”. Use ReLU activation functions after each fully connected linear module, except for the last module, after which a Tanh module should be used (constraining parameters between [-1, 1]). The number of layers and number of nodes per layer should be adjustable


In [20]:
"""
This module implements a multi-layer perceptron (MLP) in NumPy.
You should fill in code into indicated sections.
"""
from modules import MLP #you need to adapt these
from modules import MSE 




# Exercise 3C-D
C.	Train the neural network using just python and numpy code. You can find modules in "modules.py" which you will first have to fill in and train. You can then use these modules here!

You can use "do_unittest.py" to test whether the modules from "modules.py" have been implemented correctly.

You will need to add the updating of the gradients. As a sanity check, a network with 2 layers (64, 32), combined with a lr of 0.001, 30 epochs, and a batch size of 128 should work reasonably well and result in a systematic error of around 0.2% and a random error of around 20%.


In [32]:
# define parameters
hidden_layers=(64,32)
model=MLP(len(bvalues), hidden_layers, 1)
epochs = 30
learning_rate = 0.001

# initialize wandb
wandb.init(
        project="AI_for_medical_imaging", job_type="training")

loss_module1 = MSE()

# set random seed for reproducibility
torch.manual_seed(seed)

# probe available devices
if torch.cuda.is_available():  # GPU operation have separate seed
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.determinstic = True
    torch.backends.cudnn.benchmark = False

# loop over epochs
for epoch in range(epochs):
    # initiate losses to 0
    train_loss_f = 0
    val_loss_f = 0
    #loop over all training data
    for x in trainloader:
        # get data (x[0]) and put the data on the GPU if available
        batch=x[0].numpy()
        # get the reference f (x[2]) --> note x[1] and x[3] are D and Dp respectively
        f_ref = np.expand_dims(x[2].numpy(),axis=1)
        # put the data through the neural network
        f_pred = model.forward(batch)
        # calculate loss (compare predicted f to the ground trueth)
        train_loss_f += np.mean(loss_module1.forward(f_pred, f_ref))
        # propogate the loss through the network (calculate d_wights/d_loss)
        model.backward(loss_module1.backward())

        #######################
        # PUT YOUR CODE HERE  #
        #######################
        #updating of the gradients
        
        for layer in model.LM: 
            layer.weights -= learning_rate * layer.grads['weight']
            layer.bias -= learning_rate * layer.grads['bias']

        #######################
        # END OF YOUR CODE    #
        #######################
        
    # initialize error_metrics
    SD_val=0
    sys_val=0
    for x in inferloader:
        batch=x[0].numpy()
        f_ref_val = np.expand_dims(x[2].numpy(),axis=1)
        f_pred_val = model.forward(batch)
        val_loss_f += np.mean(loss_module1.forward(f_pred_val, f_ref_val))
        SD, sys = hf.error_metrics(f_pred_val,f_ref_val)
        SD_val += SD**2
        sys_val += sys
    SD_val = np.sqrt(SD_val/inferloader.__len__())
    sys_val = sys_val/inferloader.__len__()

    model.clear_cache()
    
    wandb.log({"loss/train": train_loss_f/trainloader.__len__(),"loss/val": val_loss_f/inferloader.__len__(),"error/random error":SD_val,"error/systematic error":sys_val})
    print('epoch = ' + str(epoch) + ' train loss =' + str(train_loss_f/trainloader.__len__()) +' val loss =' + str(val_loss_f/inferloader.__len__()) + 'the systematic error is ' + str(sys_val) + ' and the random error is ' + str(SD_val))

epoch = 0 train loss =0.15110451695738725 val loss =0.13967357859094745the systematic error is -0.31718562974888537 and the random error is 0.19676503107902998
epoch = 1 train loss =0.12933362420408548 val loss =0.12025642650770227the systematic error is -0.2849268648928854 and the random error is 0.19678077472318645
epoch = 2 train loss =0.1121316604476833 val loss =0.10468654363875869the systematic error is -0.2561475779295251 and the random error is 0.19678545903920672
epoch = 3 train loss =0.0983904405573263 val loss =0.09221528707486172the systematic error is -0.2305176897293819 and the random error is 0.19679031049906748
epoch = 4 train loss =0.08737897447624292 val loss =0.08222906062450137the systematic error is -0.20772855253327566 and the random error is 0.19679290461767548
epoch = 5 train loss =0.07861356029572032 val loss =0.07422724175725935the systematic error is -0.18748024761041301 and the random error is 0.19679413857179398
epoch = 6 train loss =0.07170710960158146 val

D.	It now uses the mini-batch stochastic gradient descent algorithm. Add momentum to the training update of the weights.

In [33]:
# define parameters
hidden_layers=(64,32)
model=MLP(len(bvalues), hidden_layers, 1)
epochs = 30
learning_rate = 0.001
momentum_coeff = 0.9
change_weights = {layer: np.zeros_like(layer.weights) for layer in model.LM}
change_bias = {layer: np.zeros_like(layer.bias) for layer in model.LM}

# initialize wandb
wandb.init(
        project="AI_for_medical_imaging", job_type="training")

loss_module1 = MSE()

# set random seed for reproducibility
torch.manual_seed(seed)

# probe available devices
if torch.cuda.is_available():  # GPU operation have separate seed
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.determinstic = True
    torch.backends.cudnn.benchmark = False

# loop over epochs
for epoch in range(epochs):
    # initiate losses to 0
    train_loss_f = 0
    val_loss_f = 0
    #loop over all training data
    for x in trainloader:
        # get data (x[0]) and put the data on the GPU if available
        batch=x[0].numpy()
        # get the reference f (x[2]) --> note x[1] and x[3] are D and Dp respectively
        f_ref = np.expand_dims(x[2].numpy(),axis=1)
        # put the data through the neural network
        f_pred = model.forward(batch)
        # calculate loss (compare predicted f to the ground trueth)
        train_loss_f += np.mean(loss_module1.forward(f_pred, f_ref))
        # propogate the loss through the network (calculate d_wights/d_loss)
        model.backward(loss_module1.backward())

        #######################
        # PUT YOUR CODE HERE  #
        #######################
        #updating of the gradients
        for layer in model.LM: 

            new_change_weights = learning_rate * layer.grads['weight'] + momentum_coeff * change_weights[layer]
            new_change_bias = learning_rate * layer.grads['bias'] + momentum_coeff * change_bias[layer]
            
            layer.weights -= new_change_weights
            layer.bias -= new_change_bias

            change_weights[layer] = new_change_weights
            change_bias[layer] = new_change_bias

        #######################
        # END OF YOUR CODE    #
        #######################
        
    # initialize error_metrics
    SD_val=0
    sys_val=0
    for x in inferloader:
        batch=x[0].numpy()
        f_ref_val = np.expand_dims(x[2].numpy(),axis=1)
        f_pred_val = model.forward(batch)
        val_loss_f += np.mean(loss_module1.forward(f_pred_val, f_ref_val))
        SD, sys = hf.error_metrics(f_pred_val,f_ref_val)
        SD_val += SD**2
        sys_val += sys
    SD_val = np.sqrt(SD_val/inferloader.__len__())
    sys_val = sys_val/inferloader.__len__()

    model.clear_cache()
    
    wandb.log({"loss/train": train_loss_f/trainloader.__len__(),"loss/val": val_loss_f/inferloader.__len__(),"error/random error":SD_val,"error/systematic error":sys_val})
    print('epoch = ' + str(epoch) + ' train loss =' + str(train_loss_f/trainloader.__len__()) +' val loss =' + str(val_loss_f/inferloader.__len__()) + 'the systematic error is ' + str(sys_val) + ' and the random error is ' + str(SD_val))

epoch = 0 train loss =0.10312214855702172 val loss =0.05560625150320084the systematic error is -0.12854279880753952 and the random error is 0.19680577501333985
epoch = 1 train loss =0.046750459772518065 val loss =0.04078000140690448the systematic error is -0.04121112762335953 and the random error is 0.19680234953663206
epoch = 2 train loss =0.041431747821242135 val loss =0.039372700060214934the systematic error is -0.017150636223985036 and the random error is 0.19679457852070997
epoch = 3 train loss =0.04083683882174035 val loss =0.03914080348080447the systematic error is -0.00793795470707066 and the random error is 0.19679265411365188
epoch = 4 train loss =0.040745308635939205 val loss =0.03910012618253933the systematic error is -0.00482494276550008 and the random error is 0.19679027625426668
epoch = 5 train loss =0.04085400873604819 val loss =0.03908567556208269the systematic error is -0.003515116936150102 and the random error is 0.1967813888917374
epoch = 6 train loss =0.04091161880

NOTE

To show the effect of the added momentum, we run the code from C again, only now we are using a batch-size of 16. We are then comparing the network from C with the network from D. With a smaller batch-size the effect of the momentum is more evident, since the smaller batch-sizes create more noise.

In [13]:
# set b-values at which we "measure" (i.e. simulate signal)
bvalues=[0, 10, 20, 30, 50, 75, 100, 150, 300, 500, 700, 1000]

## Set the random seeds for reproducibility
np.random.seed(seed)
random.seed(seed)

#determine
batch_size = 16

with wandb.init(project="AI_for_medical_imaging", job_type="visualize data") as run:
    data_sim = hf.sim_signal(SNR=(10,40),bvalues=bvalues,sims=10000,seed=np.random.randint(1,10000))
    # Only for visualisation purposes: here we create our "Artifact" in wandb --> this allows viewing the data in your wandb account
    for i in range(4):
        #make b-value data pairs
        example_data=[[x,y] for (x,y) in zip(bvalues,data_sim[0][i])]
        # put it in a table
        table = wandb.Table(data=example_data, columns=["b-values", "signal"])
        #tell wandb to plot the table
        wandb.log({"data_plot " + str(i): wandb.plot.scatter(table, "b-values", "signal")})

    # here we split the data into train (70%), test (15%) and validation (15%) sets
    #split = int(np.floor(len(data_sim[0]) * 0.7))
    train_set, test_set, val_set = torch.utils.data.random_split([[data_sim[0][i,:],data_sim[1][i],data_sim[2][i],data_sim[3][i]] for i in range(len(data_sim[3]))],[0.7,0.15,0.15])
    #split = int(np.floor(len(rest) * 0.5))
    #test_set, val_set = torch.utils.data.random_split([[rest[0][i,:],rest[1][i],rest[2][i],rest[3][i]] for i in range(len(rest[3]))],[split, len(rest[0]) - split])

    # train loader loads the trianing data. We want to shuffle to make sure data order is modified each epoch and different data is selected each epoch.
    trainloader = torch.utils.data.DataLoader(train_set,
                                   batch_size=batch_size,
                                   shuffle=True,
                                   drop_last=True)
    # validation data is loaded here. By not shuffling, we make sure the same data is loaded for validation every time. We can use substantially more data per batch as we are not training.
    inferloader = torch.utils.data.DataLoader(val_set,
                                   batch_size=batch_size,
                                   shuffle=False,
                                   drop_last=True)
        # validation data is loaded here. By not shuffling, we make sure the same data is loaded for validation every time. We can use substantially more data per batch as we are not training.
    testloader = torch.utils.data.DataLoader(test_set,
                                   batch_size=batch_size,
                                   shuffle=False,
                                   drop_last=True)

# define parameters
hidden_layers=(64,32)
model=MLP(len(bvalues), hidden_layers, 1)
epochs = 30
learning_rate = 0.001

# initialize wandb
wandb.init(
        project="AI_for_medical_imaging", job_type="training")

loss_module1 = MSE()

# set random seed for reproducibility
torch.manual_seed(seed)

# probe available devices
if torch.cuda.is_available():  # GPU operation have separate seed
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.determinstic = True
    torch.backends.cudnn.benchmark = False

# loop over epochs
for epoch in range(epochs):
    # initiate losses to 0
    train_loss_f = 0
    val_loss_f = 0
    #loop over all training data
    for x in trainloader:
        # get data (x[0]) and put the data on the GPU if available
        batch=x[0].numpy()
        # get the reference f (x[2]) --> note x[1] and x[3] are D and Dp respectively
        f_ref = np.expand_dims(x[2].numpy(),axis=1)
        # put the data through the neural network
        f_pred = model.forward(batch)
        # calculate loss (compare predicted f to the ground trueth)
        train_loss_f += np.mean(loss_module1.forward(f_pred, f_ref))
        # propogate the loss through the network (calculate d_wights/d_loss)
        model.backward(loss_module1.backward())

        #######################
        # PUT YOUR CODE HERE  #
        #######################
        #updating of the gradients
        
        for layer in model.LM: 
            layer.weights -= learning_rate * layer.grads['weight']
            layer.bias -= learning_rate * layer.grads['bias']

        #######################
        # END OF YOUR CODE    #
        #######################
        
    # initialize error_metrics
    SD_val=0
    sys_val=0
    for x in inferloader:
        batch=x[0].numpy()
        f_ref_val = np.expand_dims(x[2].numpy(),axis=1)
        f_pred_val = model.forward(batch)
        val_loss_f += np.mean(loss_module1.forward(f_pred_val, f_ref_val))
        SD, sys = hf.error_metrics(f_pred_val,f_ref_val)
        SD_val += SD**2
        sys_val += sys
    SD_val = np.sqrt(SD_val/inferloader.__len__())
    sys_val = sys_val/inferloader.__len__()

    model.clear_cache()
    
    wandb.log({"loss/train": train_loss_f/trainloader.__len__(),"loss/val": val_loss_f/inferloader.__len__(),"error/random error":SD_val,"error/systematic error":sys_val})
    print('epoch = ' + str(epoch) + ' train loss =' + str(train_loss_f/trainloader.__len__()) +' val loss =' + str(val_loss_f/inferloader.__len__()) + 'the systematic error is ' + str(sys_val) + ' and the random error is ' + str(SD_val))



error/random error,████████████████████▇▇▆▅▃▁▁▁█████████▆▄▁
error/systematic error,▁▄▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇▆▆▆▇▇▇▇▇▇█▇██▇▆▇▆▆
loss/train,█▄▄▃▃▃▃▃▃▃▃▃▃▃▄▃▃▃▃▃▃▃▃▃▃▁▃▃▃▃▃▃▃▃▃▃▃▂▂▁
loss/val,█▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▅▄▃▂▁▁▅▅▅▅▅▅▅▅▄▃▃▂▁
error/random error,0.0981
error/systematic error,-0.00484
loss/train,0.01056
loss/val,0.0103


epoch = 0 train loss =0.09898557806678354 val loss =0.06241541259889638the systematic error is -0.1493571862058027 and the random error is 0.1951254733695247
epoch = 1 train loss =0.05206255856947782 val loss =0.04484842332759573the systematic error is -0.06885281683604144 and the random error is 0.19512516733504306
epoch = 2 train loss =0.04344828695544876 val loss =0.04123015266757754the systematic error is -0.03353883426699769 and the random error is 0.1951192484291124
epoch = 3 train loss =0.041597296040805205 val loss =0.040384686978869606the systematic error is -0.016796475396452955 and the random error is 0.19511263655667765
epoch = 4 train loss =0.04118826418239022 val loss =0.040183969804450374the systematic error is -0.009226126518475855 and the random error is 0.19510353922728166
epoch = 5 train loss =0.04107053000543822 val loss =0.04012336882422833the systematic error is -0.005231887604367496 and the random error is 0.19509652088939053
epoch = 6 train loss =0.0410224139607

In [14]:
# define parameters
hidden_layers=(64,32)
model=MLP(len(bvalues), hidden_layers, 1)
epochs = 30
learning_rate = 0.001
momentum_coeff = 0.9
change_weights = {layer: np.zeros_like(layer.weights) for layer in model.LM}
change_bias = {layer: np.zeros_like(layer.bias) for layer in model.LM}

# initialize wandb
wandb.init(
        project="AI_for_medical_imaging", job_type="training")

loss_module1 = MSE()

# set random seed for reproducibility
torch.manual_seed(seed)

# probe available devices
if torch.cuda.is_available():  # GPU operation have separate seed
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.determinstic = True
    torch.backends.cudnn.benchmark = False

# loop over epochs
for epoch in range(epochs):
    # initiate losses to 0
    train_loss_f = 0
    val_loss_f = 0
    #loop over all training data
    for x in trainloader:
        # get data (x[0]) and put the data on the GPU if available
        batch=x[0].numpy()
        # get the reference f (x[2]) --> note x[1] and x[3] are D and Dp respectively
        f_ref = np.expand_dims(x[2].numpy(),axis=1)
        # put the data through the neural network
        f_pred = model.forward(batch)
        # calculate loss (compare predicted f to the ground trueth)
        train_loss_f += np.mean(loss_module1.forward(f_pred, f_ref))
        # propogate the loss through the network (calculate d_wights/d_loss)
        model.backward(loss_module1.backward())

        #######################
        # PUT YOUR CODE HERE  #
        #######################
        #updating of the gradients
        for layer in model.LM: 

            new_change_weights = learning_rate * layer.grads['weight'] + momentum_coeff * change_weights[layer]
            new_change_bias = learning_rate * layer.grads['bias'] + momentum_coeff * change_bias[layer]
            
            layer.weights -= new_change_weights
            layer.bias -= new_change_bias

            change_weights[layer] = new_change_weights
            change_bias[layer] = new_change_bias

        #######################
        # END OF YOUR CODE    #
        #######################
        
    # initialize error_metrics
    SD_val=0
    sys_val=0
    for x in inferloader:
        batch=x[0].numpy()
        f_ref_val = np.expand_dims(x[2].numpy(),axis=1)
        f_pred_val = model.forward(batch)
        val_loss_f += np.mean(loss_module1.forward(f_pred_val, f_ref_val))
        SD, sys = hf.error_metrics(f_pred_val,f_ref_val)
        SD_val += SD**2
        sys_val += sys
    SD_val = np.sqrt(SD_val/inferloader.__len__())
    sys_val = sys_val/inferloader.__len__()

    model.clear_cache()
    
    wandb.log({"loss/train": train_loss_f/trainloader.__len__(),"loss/val": val_loss_f/inferloader.__len__(),"error/random error":SD_val,"error/systematic error":sys_val})
    print('epoch = ' + str(epoch) + ' train loss =' + str(train_loss_f/trainloader.__len__()) +' val loss =' + str(val_loss_f/inferloader.__len__()) + 'the systematic error is ' + str(sys_val) + ' and the random error is ' + str(SD_val))

epoch = 0 train loss =0.05000394705769913 val loss =0.04008040137444191the systematic error is -0.0023732136385089097 and the random error is 0.19504413882891897
epoch = 1 train loss =0.04104263738717604 val loss =0.04007766593069095the systematic error is -0.0035085891982727427 and the random error is 0.1950210572465974
epoch = 2 train loss =0.041006072424439675 val loss =0.04005715380087678the systematic error is 0.0012300790165032931 and the random error is 0.19499722607433415
epoch = 3 train loss =0.041009178050058596 val loss =0.0400444057268614the systematic error is -6.106895087787718e-05 and the random error is 0.1949696498783934
epoch = 4 train loss =0.04100744717389617 val loss =0.040048754692404494the systematic error is -0.004156552017776363 and the random error is 0.19493793446110974
epoch = 5 train loss =0.04097087518779344 val loss =0.040024930067486746the systematic error is 0.0026931107184629097 and the random error is 0.19490405600050603
epoch = 6 train loss =0.040932